In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_grouped = pd.read_csv('df_grouped.csv')

In [ ]:
# 1. Выбираем 1000 случайных чатов с ошибкой
df_error = df_grouped[df_grouped['error'] == 1].sample(n=500, random_state=42)

# 2. Выбираем 1000 случайных чатов без ошибки
df_no_error = df_grouped[df_grouped['error'] == 0].sample(n=500, random_state=42)

# Объединяем обе выборки в один датафрейм
df_gpt = pd.concat([df_error, df_no_error]).sample(frac=1, random_state=42).reset_index(drop=True)

print(df_gpt)

### GPT 4o zero shot

In [ ]:
import pandas as pd
import requests
import time
import re
from sklearn.metrics import classification_report

# --- НАСТРОЙКИ API ---
AUTH_URL = '...'
PROXY_URL = '...'
MODEL = 'gpt-4o'

# --- Авторизация ---
def get_access_token(username: str, password: str) -> str:
    data = {"username": username, "password": password}
    response = requests.post(AUTH_URL, json=data)
    response.raise_for_status()
    return response.json()['access_token']

# --- Отправка одного запроса ---
def send_prompt(prompt: str, access_token: str) -> str:
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'x-proxy-mask-critical-data': '1'
    }
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(PROXY_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']

# --- Формирование промпта ---
def make_prompt(chat_text: str) -> str:
    return (
        "Ниже приведен чат между клиентом и оператором службы поддержки банка. "
        "Определи, допустил ли оператор ошибку: **Не сгладил негатив клиента**. "
        "Чат состоит из нескольких сообщений. Перед началом сообщения указано от кого оно. "
        "Проверь чат на наличие ошибки и ответь строго 0 (если считаешь что в чате нет ошибки) или 1 (если считаешь что ошибка была).\n\n"
        "Чат:\n"
        f"{chat_text}\n\n"
        "Ответ:"
    )

# --- Безопасное извлечение ответа 0 или 1 ---
def extract_prediction(response: str) -> int:
    match = re.search(r'\b[01]\b', response)
    if match:
        return int(match.group())
    return None  # если нет нормального ответа

# --- Основная функция ---
def evaluate_gpt(df_gpt, username, password, limit=None, delay=1.5):
    token = get_access_token(username, password)
    true_labels = []
    predicted_labels = []
    skipped_chats = 0  # счётчик пропущенных чатов

    for i, row in df_gpt.iterrows():
        if limit and i >= limit:
            break
        
        prompt = make_prompt(row['message_txt'])

        try:
            response = send_prompt(prompt, token)
            prediction = extract_prediction(response)

            # Только если извлечён валидный ответ (0 или 1)
            if prediction is not None:
                true_labels.append(row['error'])
                predicted_labels.append(prediction)
                print(f"[{i+1}/{len(df_gpt)}] true={row['error']} | predicted={prediction}")
            else:
                print(f" Невалидный ответ от GPT на чате {i}, чат пропущен.")
                skipped_chats += 1

        except Exception as e:
            print(f" Ошибка на чате {i}: {e}")
            skipped_chats += 1
            continue

        time.sleep(delay)

    if not true_labels:
        print("\n Нет успешно обработанных чатов. Метрики посчитать невозможно.")
        return pd.DataFrame()

    # Выводим метрики по каждому классу
    print("\n=== Classification Report ===")
    print(classification_report(true_labels, predicted_labels, digits=4, target_names=["no_error", "error"]))

    print(f"\nПропущено чатов из-за ошибок или невалидных ответов: {skipped_chats}")

    return pd.DataFrame({
        'true_error': true_labels,
        'predicted_error': predicted_labels
    })

# --- Пример использования ---
if __name__ == '__main__':
    username = Z_ENV_LOGIN_GPT  # или os.getenv("Z_ENV_LOGIN_GPT")
    password = Z_ENV_PASS_GPT   # или os.getenv("Z_ENV_PASS_GPT")

    results_df = evaluate_gpt(df_gpt, username, password)

### GPT o3 few shot

In [ ]:
import pandas as pd
import requests
import time
import re
from sklearn.metrics import classification_report

# --- НАСТРОЙКИ API ---
AUTH_URL = 'https://openai-proxy.tcsbank.ru/auth/v1/token'
PROXY_URL = 'https://openai-proxy.tcsbank.ru/public/v1/chat/completions'
MODEL = 'o3'

# --- Авторизация ---
def get_access_token(username: str, password: str) -> str:
    data = {"username": username, "password": password}
    response = requests.post(AUTH_URL, json=data)
    response.raise_for_status()
    return response.json()['access_token']

# --- Отправка одного запроса ---
def send_prompt(prompt: str, access_token: str) -> str:
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'x-proxy-mask-critical-data': '1'
    }
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(PROXY_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']

# --- Формирование промпта ---
def make_prompt(chat_text: str) -> str:
    return (
        "Ниже приведен чат между клиентом и оператором службы поддержки банка. "
        "Определи, допустил ли оператор ошибку: **Не сгладил негатив клиента**. "
        "Чат состоит из нескольких сообщений. Перед началом сообщения указано от кого оно. "
        "Проверь чат на наличие ошибки и ответь строго 0 (если считаешь что в чате нет ошибки) или 1 (если считаешь что ошибка была).\n\n"
        "Чат:\n"
        f"{chat_text}\n\n"
        "Ответ:"
    )

# --- Безопасное извлечение ответа 0 или 1 ---
def extract_prediction(response: str) -> int:
    match = re.search(r'\b[01]\b', response)
    if match:
        return int(match.group())
    return None  # если нет нормального ответа

# --- Основная функция ---
def evaluate_gpt(df_gpt, username, password, limit=None, delay=1.5):
    token = get_access_token(username, password)
    true_labels = []
    predicted_labels = []
    skipped_chats = 0  # счётчик пропущенных чатов

    for i, row in df_gpt.iterrows():
        if limit and i >= limit:
            break
        
        prompt = make_prompt(row['message_txt'])

        try:
            response = send_prompt(prompt, token)
            prediction = extract_prediction(response)

            # Только если извлечён валидный ответ (0 или 1)
            if prediction is not None:
                true_labels.append(row['error'])
                predicted_labels.append(prediction)
                print(f"[{i+1}/{len(df_gpt)}] true={row['error']} | predicted={prediction}")
            else:
                print(f"⚠️ Невалидный ответ от GPT на чате {i}, чат пропущен.")
                skipped_chats += 1

        except Exception as e:
            print(f"❌ Ошибка на чате {i}: {e}")
            skipped_chats += 1
            continue

        time.sleep(delay)

    if not true_labels:
        print("\n⚠️ Нет успешно обработанных чатов. Метрики посчитать невозможно.")
        return pd.DataFrame()

    # Выводим метрики по каждому классу
    print("\n=== Classification Report ===")
    print(classification_report(true_labels, predicted_labels, digits=4, target_names=["no_error", "error"]))

    print(f"\nПропущено чатов из-за ошибок или невалидных ответов: {skipped_chats}")

    return pd.DataFrame({
        'true_error': true_labels,
        'predicted_error': predicted_labels
    })

# --- Пример использования ---
if __name__ == '__main__':
    username = Z_ENV_LOGIN_GPT  # или os.getenv("Z_ENV_LOGIN_GPT")
    password = Z_ENV_PASS_GPT   # или os.getenv("Z_ENV_PASS_GPT")

    results_df = evaluate_gpt(df_gpt, username, password)

### GPT 4o few shot

In [ ]:
import pandas as pd
import requests
import time
import re
from sklearn.metrics import classification_report

# --- НАСТРОЙКИ API ---
AUTH_URL = '...'
PROXY_URL = '...'
MODEL = 'gpt-4o'

# --- Авторизация ---
def get_access_token(username: str, password: str) -> str:
    data = {"username": username, "password": password}
    response = requests.post(AUTH_URL, json=data)
    response.raise_for_status()
    return response.json()['access_token']

# --- Отправка одного запроса ---
def send_prompt(prompt: str, access_token: str) -> str:
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'x-proxy-mask-critical-data': '1'
    }
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(PROXY_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']

# --- Формирование промпта ---
def make_prompt(chat_text: str) -> str:
    return (
        "You are the world’s most accurate expert in analyzing customer support chats for emotional intelligence errors. "
        "Your specific task is to detect whether the support operator **failed to defuse the customer's negativity** during the interaction.\n\n"
        "Definition of the error:\n"
        "- The operator does **not** apologize when the customer is clearly upset or frustrated.\n"
        "- The operator does **not** attempt to solve the problem or reassure the customer.\n"
        "- The operator shows **no empathy** when the customer expresses dissatisfaction.\n\n"
        "- or the operator does **not** smooth a client's negative any other way.\n\n"
        "Examples:\n"
        "- <client> Ненавижу ваш банк! Где мой кэшбек?? <operator> Придет в дату выписки. - The operator answered, but did **not** smooth the customer's negative about a bank - it is a mistake.\n"
        "- <client> Почему мне отказали в кредитке?? Терпеть вас не могу. <operator> Ни в коем случае не хотели вас расстроить! Понимаю, как важно иметь под рукой кредитную карту. Мы пересмотрим наше решение. - The operator smoothed the client's negative, apologizing for the situation - there is no mistake.\n"
        "Instructions:\n"
        "- Analyze the full conversation (given on Russian language) carefully.\n"
        "- If the operator **makes no effort** to calm the customer when there is visible negativity (according to instructions), return `1`.\n"
        "- If the operator **acknowledges the customer's frustration**, tries to smooth his negative, or there is no negative from client at all, return `0`.\n\n"
        "Return only a single digit: `1` (error detected) or `0` (no error).\n\n"
        "Chat:\n"
        f"{chat_text}\n\n"
        "Your answer:"
    )

# --- Безопасное извлечение ответа 0 или 1 ---
def extract_prediction(response: str) -> int:
    match = re.search(r'\b[01]\b', response)
    if match:
        return int(match.group())
    return None  # если нет нормального ответа

# --- Основная функция ---
def evaluate_gpt(df_gpt, username, password, limit=None, delay=1.5):
    token = get_access_token(username, password)
    true_labels = []
    predicted_labels = []
    skipped_chats = 0  # счётчик пропущенных чатов

    for i, row in df_gpt.iterrows():
        if limit and i >= limit:
            break
        
        prompt = make_prompt(row['message_txt'])

        try:
            response = send_prompt(prompt, token)
            prediction = extract_prediction(response)

            # Только если извлечён валидный ответ (0 или 1)
            if prediction is not None:
                true_labels.append(row['error'])
                predicted_labels.append(prediction)
                print(f"[{i+1}/{len(df_gpt)}] true={row['error']} | predicted={prediction}")
            else:
                print(f"⚠️ Невалидный ответ от GPT на чате {i}, чат пропущен.")
                skipped_chats += 1

        except Exception as e:
            print(f"❌ Ошибка на чате {i}: {e}")
            skipped_chats += 1
            continue

        time.sleep(delay)

    if not true_labels:
        print("\n⚠️ Нет успешно обработанных чатов. Метрики посчитать невозможно.")
        return pd.DataFrame()

    # Выводим метрики по каждому классу
    print("\n=== Classification Report ===")
    print(classification_report(true_labels, predicted_labels, digits=4, target_names=["no_error", "error"]))

    print(f"\nПропущено чатов из-за ошибок или невалидных ответов: {skipped_chats}")

    return pd.DataFrame({
        'true_error': true_labels,
        'predicted_error': predicted_labels
    })

# --- Пример использования ---
if __name__ == '__main__':
    username = Z_ENV_LOGIN_GPT  # или os.getenv("Z_ENV_LOGIN_GPT")
    password = Z_ENV_PASS_GPT   # или os.getenv("Z_ENV_PASS_GPT")

    results_df = evaluate_gpt(df_gpt, username, password)

In [ ]:
import pandas as pd
import requests
import time
import re
from sklearn.metrics import classification_report

# --- НАСТРОЙКИ API ---
AUTH_URL = 'https://openai-proxy.tcsbank.ru/auth/v1/token'
PROXY_URL = 'https://openai-proxy.tcsbank.ru/public/v1/chat/completions'
MODEL = 'o3'

# --- Авторизация ---
def get_access_token(username: str, password: str) -> str:
    data = {"username": username, "password": password}
    response = requests.post(AUTH_URL, json=data)
    response.raise_for_status()
    return response.json()['access_token']

# --- Отправка одного запроса ---
def send_prompt(prompt: str, access_token: str) -> str:
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'x-proxy-mask-critical-data': '1'
    }
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(PROXY_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']

# --- Формирование промпта ---
def make_prompt(chat_text: str) -> str:
    return (
        "You are the world’s most accurate expert in analyzing customer support chats for emotional intelligence errors. "
        "Your specific task is to detect whether the support operator **failed to defuse the customer's negativity** during the interaction.\n\n"
        "Definition of the error:\n"
        "- The operator does **not** apologize when the customer is clearly upset or frustrated.\n"
        "- The operator does **not** attempt to solve the problem or reassure the customer.\n"
        "- The operator shows **no empathy** when the customer expresses dissatisfaction.\n\n"
        "- or the operator does **not** smooth a client's negative any other way.\n\n"
        "Examples:\n"
        "- <client> Ненавижу ваш банк! Где мой кэшбек?? <operator> Придет в дату выписки. - The operator answered, but did **not** smooth the customer's negative about a bank - it is a mistake.\n"
        "- <client> Почему мне отказали в кредитке?? Терпеть вас не могу. <operator> Ни в коем случае не хотели вас расстроить! Понимаю, как важно иметь под рукой кредитную карту. Мы пересмотрим наше решение. - The operator smoothed the client's negative, apologizing for the situation - there is no mistake.\n"
        "Instructions:\n"
        "- Analyze the full conversation (given on Russian language) carefully.\n"
        "- If the operator **makes no effort** to calm the customer when there is visible negativity (according to instructions), return `1`.\n"
        "- If the operator **acknowledges the customer's frustration**, tries to smooth his negative, or there is no negative from client at all, return `0`.\n\n"
        "Return only a single digit: `1` (error detected) or `0` (no error).\n\n"
        "Chat:\n"
        f"{chat_text}\n\n"
        "Your answer:"
    )

# --- Безопасное извлечение ответа 0 или 1 ---
def extract_prediction(response: str) -> int:
    match = re.search(r'\b[01]\b', response)
    if match:
        return int(match.group())
    return None  # если нет нормального ответа

# --- Основная функция ---
def evaluate_gpt(df_gpt, username, password, limit=None, delay=1.5):
    token = get_access_token(username, password)
    true_labels = []
    predicted_labels = []
    skipped_chats = 0  # счётчик пропущенных чатов

    for i, row in df_gpt.iterrows():
        if limit and i >= limit:
            break
        
        prompt = make_prompt(row['message_txt'])

        try:
            response = send_prompt(prompt, token)
            prediction = extract_prediction(response)

            # Только если извлечён валидный ответ (0 или 1)
            if prediction is not None:
                true_labels.append(row['error'])
                predicted_labels.append(prediction)
                print(f"[{i+1}/{len(df_gpt)}] true={row['error']} | predicted={prediction}")
            else:
                print(f"⚠️ Невалидный ответ от GPT на чате {i}, чат пропущен.")
                skipped_chats += 1

        except Exception as e:
            print(f"❌ Ошибка на чате {i}: {e}")
            skipped_chats += 1
            continue

        time.sleep(delay)

    if not true_labels:
        print("\n⚠️ Нет успешно обработанных чатов. Метрики посчитать невозможно.")
        return pd.DataFrame()

    # Выводим метрики по каждому классу
    print("\n=== Classification Report ===")
    print(classification_report(true_labels, predicted_labels, digits=4, target_names=["no_error", "error"]))

    print(f"\nПропущено чатов из-за ошибок или невалидных ответов: {skipped_chats}")

    return pd.DataFrame({
        'true_error': true_labels,
        'predicted_error': predicted_labels
    })

# --- Пример использования ---
if __name__ == '__main__':
    username = Z_ENV_LOGIN_GPT  # или os.getenv("Z_ENV_LOGIN_GPT")
    password = Z_ENV_PASS_GPT   # или os.getenv("Z_ENV_PASS_GPT")

    results_df = evaluate_gpt(df_gpt, username, password)